<h1>Load Required Functions</h1>

In [5]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

BUFFER_SIZE = 100
WIDTH = 1280
HEIGHT = 720
video_count = 0

In [6]:
def calculate_n_of_hsv(frame_sq,n):
    sum_n = 0
    var_n = 0
    for frame in frame_sq:
        var_n += np.var(frame[:,:,n])# / (np.mean(frame[:,:,n]) * 1
        sum_n += np.sum(frame[:,:,n])

    avg_n = sum_n/ (len(frame_sq) * frame_sq[0].shape[0] * frame_sq[0].shape[1])
    return np.array([avg_n ,var_n])
def calculate_comp(frame_sq):
    sum_comp = 0
    for i in range(len(frame_sq)):
        if(i != 0):
            sum_comp += compare_img(frame_sq[i], frame_sq[i-1])
#         if(i == 1): print(compare_img(frame_sq[i], frame_sq[i-1]))
    sum_comp /= (len(frame_sq) - 1)
    return sum_comp
def calculate_opticFlow(frame_sq):
    prvs = cv2.resize(frame_sq[0],(0,0),fx=0.5,fy=0.5)
    mag_sum = 0
    ang_sum = 0
    var_mag = 0
    var_ang = 0
    for i in range(1,len(frame_sq)):
        frame2 = frame_sq[i]
        next = cv2.resize(frame2, (0,0), fx=0.5, fy=0.5) 
        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        mag = cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
        ang = np.absolute(cv2.normalize(ang,None,-1,1,cv2.NORM_MINMAX))
        mag_sum = np.sum(mag)
        ang_sum = np.sum(ang)
        var_mag += np.var(mag)
        var_ang += np.var(ang)
    mag_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    ang_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    var_mag /= BUFFER_SIZE
    var_ang /= BUFFER_SIZE
    return np.array([mag_sum, ang_sum, var_mag, var_ang])

In [7]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
    data.remove(data[0])
    for datum in data:
        dataset = np.append(dataset, datum, axis = 1)
    return dataset
def createTSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "dataset" + str(dataset_no) + ".tsv"
    datasetName = filename + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "label" + str(dataset_no) + ".tsv"
    datasetName = filename
    file = open(datasetName, "w")
    file.write("havg\thv\tsavg\tsv\tvavg\tvv\tmagavg\tmagv\tangavg\tangv\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return


In [9]:
from sklearn.preprocessing import StandardScaler
from numpy import genfromtxt

dataset = (genfromtxt("../normFrankensteinFullForm.csv", delimiter=','))[1:,:]
scaler = StandardScaler(copy=True)
print(scaler.fit(dataset))

def normalize(frame):
    frame = scaler.transform(frame)
    return frame
#     datasetNan = np.isnan(dataset).any(axis=1)
#     nans = []
#     for i in range(datasetNan.shape[0]):
#         if(datasetNan[i] == True):
#             nans.append(i)
#     dataset = dataset[~np.isnan(dataset).any(axis=1)]
#     max_values = np.max(dataset,axis=0)
#     min_values = np.min(dataset,axis=0)
#     ranges = max_values - min_values
#     dataset /= ranges
    return (dataset,nans)
    

StandardScaler(copy=True, with_mean=True, with_std=True)


<h1>Load Model</h1>

In [10]:
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model4.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights("model4.h5")
print("Loaded model from disk")


C:\Users\Bone\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Loaded model from disk


In [11]:
from numpy import genfromtxt
dataset = (genfromtxt("./FrankensteinFullForm.csv", delimiter=','))[1:,:]

In [12]:
1.68698948e-01 + 3

3.168698948

In [13]:
_min = [1.68698948e-01,5.71359754e-01,7.26006841e-02,1.91583500e-01,2.30490743e-01,1.69076206e+00,1.60691440e-06,3.60041682e-04,7.00276490e-04,5.24942415e-02]
_range = [1.88006912e-01, 2.37081875e+00, 5.25229358e-01, 1.84091510e+01,3.98871483e-01, 1.38006918e+01, 9.30031806e-05,1.20997849e-03,4.86319761e-03,7.18616676e-02]
def process(frame_sq,frame_sq_gray):
    h = calculate_n_of_hsv(frame_sq,0)
    s = calculate_n_of_hsv(frame_sq,1)
    v = calculate_n_of_hsv(frame_sq,2)
    opticFlow = calculate_opticFlow(frame_sq_gray)
    return normalize(np.append(h,np.append(s,np.append(v,opticFlow))))
def normalize(dataset):
    datasetTmp = np.copy(dataset)
    datasetTmp -= _min
    datasetTmp /= _range
    return datasetTmp

In [14]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
def predict(output):
    prediction = model.predict(output.reshape(1,10))
    return(definition[np.argmax(prediction[0])])

<h1>Start</h1>

In [15]:
import numpy as np
import cv2
import sys
from sklearn import preprocessing

frame_sq = []
frame_sq_gray = []
BUFFER_SIZE = 100
cap = cv2.VideoCapture(1)
video_count = 0
while(True):
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    if(video_count == 0):
        WIDTH = int(cap.get(3))
        HEIGHT = int(cap.get(4))
        video_count += 1
    frame_sq.append(hsv)
    frame_sq_gray.append(gray)
    if(len(frame_sq) + 1 >= BUFFER_SIZE):
        output = process(frame_sq, frame_sq_gray)
        print(output)
        predictt = predict(output)
        sys.stdout.write("\r" + predictt)
        sys.stdout.flush()        
#         del frame_sq[0]
        frame_sq = []
        frame_sq_gray = []
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

[2.86771378e+02 4.71243260e+04 1.32189193e+02 2.43635850e+04
 2.83951730e+02 2.84082168e+04 1.23239958e+00 6.32137155e-01
            nan 3.61426508e-01]
static[3.06564511e+02 5.68965921e+04 1.02362994e+02 1.76482700e+04
 2.66715803e+02 2.34763566e+04 1.70859928e+00 7.69585453e-01
 2.52142885e+00 5.49537900e-01]
light[2.92185921e+02 5.98401686e+04 1.23429093e+02 2.47662582e+04
 2.95236490e+02 3.90500800e+04 2.03809941e+00 7.04094554e-01
 7.77712948e+00 4.52255486e-01]
light[2.62040175e+02 5.05976739e+04 1.51046673e+02 2.89660497e+04
 3.05567857e+02 3.87913757e+04 2.63318260e+00 7.08621093e-01
 6.08020913e+00 7.20169452e-01]
light[2.59347337e+02 6.80913217e+04 5.58835145e+01 6.01822520e+03
 3.12085694e+02 1.64292115e+04 2.48101266e+00 7.21628496e-01
 3.39694352e+00 3.56911166e-01]
light[2.50214880e+02 7.03754923e+04 5.56481714e+01 6.12173883e+03
 3.06099069e+02 1.63708488e+04 1.36426062e+00 7.97150020e-01
 3.12299792e+00 4.92270790e-01]
light